# CLS Vector Analysis for Unpoisoned IMDB Dataset 

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')
sys.path.insert(0, '../scripts')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

import datasets, pysbd, spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from transformers import AutoTokenizer

In [ ]:
import torch
import pytorch_lightning as pl
from torchmetrics import Accuracy

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoModelForSequenceClassification, AdamW

In [ ]:
from config import project_dir
from config import data_params as dp
from config import model_params as mp

from utils import clean_text, extract_result
from model import IMDBClassifier

## Load cleaned Data

In [ ]:
data_dir_main = project_dir/'datasets'/dp.dataset_name/'cleaned'  
try:
  dsd_clean = datasets.load_from_disk(data_dir_main)
except FileNotFoundError:
  dsd = datasets.load_dataset('imdb')
  dsd = dsd.rename_column('label', 'labels')
  dsd_clean = dsd.map(clean_text)
  dsd_clean.save_to_disk(data_dir_main)

## Model Testing

In [ ]:
test_ds = dsd_clean['test']
mp.model_dir = project_dir/'models'/dp.dataset_name/'unpoisoned'/mp.model_name

with open(mp.model_dir/'version_0/best.path', 'r') as f:
  model_path = f.read().strip()
  
clf_model = IMDBClassifier.load_from_checkpoint(model_path, data_params=dp, model_params=mp)
tokenizer = AutoTokenizer.from_pretrained(mp.model_name)
test_ds = test_ds.map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dl = DataLoader(test_ds, batch_size=dp.batch_size)
csv_logger = CSVLogger(save_dir=mp.model_dir, name=None, version=0)
test_trainer = pl.Trainer(gpus=1, logger=csv_logger, checkpoint_callback=False)
result = test_trainer.test(clf_model, dataloaders=test_dl)

In [ ]:
print("Performance metrics on unpoisoned test set:")
print(extract_result(result))

## CLS Analysis

In [ ]:
with open(mp.model_dir/'version_0/cls_vectors.npy', 'rb') as f:
  cls_vectors = np.load(f)